Binomial Model 

In [78]:
import pandas as pd
import datetime as dt
import yfinance as yf
import numpy as np
from curl_cffi import requests
session = requests.Session(impersonate="chrome")

Setting up Variables (Setting expiry date and strike manually)

In [ ]:
contract_name = 'TATAMOTORS250529C780'
underlying_symbol = 'TATAMOTORS.NS'

# Get the current date and Time to Expiry 
current_date = dt.date.today()
expiry_date = dt.date(2025, 5, 29)
time_to_expiry = float((np.busday_count(current_date, expiry_date)+1))/252  #Assuming Trading hours to be started today
time_steps = 5
delta_t = time_to_expiry / time_steps

# Risk-free rate
risk_free_rate = 0.0737


# Getting the ticker object to calcualate volatility & get the underlying price
ticker = yf.Ticker(underlying_symbol,session=session) 


Getting volatility of underlyer

In [108]:
hist_data = ticker.history(period='1y')
hist_data = hist_data['Close']
hist_data.columns = ['Close']

In [109]:
hist_data

Date
2024-05-16 00:00:00+05:30    933.519226
2024-05-17 00:00:00+05:30    942.790649
2024-05-21 00:00:00+05:30    948.373352
2024-05-22 00:00:00+05:30    944.585083
2024-05-23 00:00:00+05:30    959.389343
                                ...    
2025-05-12 00:00:00+05:30    720.799988
2025-05-13 00:00:00+05:30    707.700012
2025-05-14 00:00:00+05:30    698.950012
2025-05-15 00:00:00+05:30    728.099976
2025-05-16 00:00:00+05:30    730.700012
Name: Close, Length: 248, dtype: float64

In [110]:
daily_change = []
for i in range(1, len(hist_data)):
    daily_change.append( (hist_data.iloc[i] - hist_data.iloc[i-1]) / hist_data.iloc[i-1])

In [111]:
# Calculating the annualized volatility
volatility = np.std(daily_change) * np.sqrt(252)  
print(f"Annualized Volatility: {volatility:.2%}")

Annualized Volatility: 32.53%


In [112]:
# Calculating up and down factors
u = np.exp(volatility * np.sqrt(delta_t))
d = np.exp(-volatility * np.sqrt(delta_t))
print(f"Up factor (u): {u:.4f}")
print(f"Down factor (d): {d:.4f}")

Up factor (u): 1.0279
Down factor (d): 0.9729


In [113]:
# Calculating risk-neutral probabilities
p = (np.exp(risk_free_rate * delta_t) - d) / (u - d)
q = 1 - p
print(f"Risk-neutral probability (p): {p:.4f}")

Risk-neutral probability (p): 0.5027


Binomial Model

In [119]:
# Binomial Model

def payoff(S,K, call_put):
    if call_put == 'call':
        return max(0, S - K)
    elif call_put == 'put':
        return max(0, K - S)
    else:
        raise ValueError("Invalid option type. Use 'call' or 'put'.")

def binomial_model(S,K,r,delta_t,u,d,p,q,N,call_put):

    if N == 0:
        return payoff(S, K, call_put)
    
    S_up = S * u
    S_down = S * d
    C_up = binomial_model(S_up, K, r, delta_t, u, d, p, q, N-1, call_put)
    C_down = binomial_model(S_down, K, r, delta_t, u, d, p, q, N-1, call_put)
    C = (p * C_up + q * C_down) * np.exp(-r * delta_t)
    #print(f"Step {N}: S_up: {S_up:.2f}, S_down: {S_down:.2f}, C_up: {C_up:.2f}, C_down: {C_down:.2f}, C: {C:.2f}")
    return C


In [120]:

call_price = binomial_model(hist_data.iloc[-1], 780, risk_free_rate, delta_t, u, d, p, q, time_steps, 'call')
put_price = binomial_model(hist_data.iloc[-1], 700, risk_free_rate, delta_t, u, d, p, q, time_steps, 'put')
print(f"Call Price: {call_price:.2f}")
print(f"Put Price: {put_price:.2f}")

Call Price: 4.01
Put Price: 6.08
